In [1]:
!pip3 install pykakasi
!pip3 install googletrans

     |████████████████████████████████| 2.4MB 2.7MB/s 
     |████████████████████████████████| 184kB 26.6MB/s 
     |████████████████████████████████| 133kB 21.0MB/s 
  Created wheel for klepto: filename=klepto-0.1.9-cp36-none-any.whl size=83059 sha256=a741c4b0798d7f4371bc4687038118c3b34db8311914095738a39681aa113f4c
  Stored in directory: /root/.cache/pip/wheels/e4/fd/04/228e9d009dc68ea735f04a47f1ae24a945fdd45ac9ba0a1525
  Created wheel for pox: filename=pox-0.2.8-cp36-none-any.whl size=28290 sha256=ebc1dd8d4a3f16efe8a646c08ebe2c4c63ab07ebbd1fcfd785af168b3fcc055b
  Stored in directory: /root/.cache/pip/wheels/39/ed/ce/a93103746b327e18bffaeb99ba0d57a88b392f31d719cea700
Successfully built klepto pox
     |████████████████████████████████| 61kB 2.7MB/s 
     |████████████████████████████████| 972kB 8.2MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 71kB 7.1MB/s 
     |████████████████

In [2]:
!tar xvf lyric-data.tar.gz

lyric-data/
lyric-data/Okujou-kara-sakebu.txt
lyric-data/Kimi-wa-boku-no-kaze.txt
lyric-data/Choco-no-yukue.txt
lyric-data/Kokoro-no-kusuri.txt
lyric-data/Dance-wo-omoidasu-made.txt
lyric-data/Aitakatta-AKB48-SingleAlbum-Version.txt
lyric-data/Dakara-kimi-ga-suki-na-no-ka.txt
lyric-data/Boku-no-koto,-shitteru.txt
lyric-data/13nichi-no-kinyoubi.txt
lyric-data/Tenshi-ni-wa-narenai.txt
lyric-data/You-Should-Try-HARDer.txt
lyric-data/Ishikari-banka.txt
lyric-data/Haru-no-hikari-chikadzuita-natsu.txt
lyric-data/Itsumo-soba-ni.txt
lyric-data/Baby-Baby-Baby.txt
lyric-data/Lavender-no-Juutan.txt
lyric-data/Futsuu-no-anata.txt
lyric-data/Koi-suru-Fortune-Cookie-AKB48Iwasa-Misaki-Single-Version.txt
lyric-data/Be-happy.txt
lyric-data/Pinocchio-gun-SKE48-Stage-Version.txt
lyric-data/Wanna-be-now.txt
lyric-data/Asu-mo-waraou.txt
lyric-data/Yume-no-nagori.txt
lyric-data/Bokura-no-shunkashuutou.txt
lyric-data/Shoujotachi-yo-NO-NAME-Single-Version.txt
lyric-data/7ji-12fun-no-hatsukoi.txt
lyric-data/Pa

In [3]:
from googletrans import Translator
translator = Translator()

In [4]:
import requests
import bs4
from bs4 import BeautifulSoup 
import pykakasi
import re
import math

In [5]:
#URL = "http://stage48.net/studio48/1gallonnoase.html"
URL = "http://stage48.net/studio48/areak.html"

In [6]:
kks = pykakasi.kakasi()

In [7]:

r = requests.get(URL) 
   
soup = BeautifulSoup(r.content, 'html5lib') 
   
quotes=[]  # a list to store quotes
   
table = soup.find('div', attrs = {'class':'content'})  

In [ ]:
def lyricProcess(text):
  text = text.strip()
  #text = text.replace(' ','')
  text = text.replace('  ','')
  text = text.replace('	','')

  #text = text.replace('\n','')
  
  return text

def convertToKana(text):
  resStr = ''
  for kanaStr in kks.convert(text):
    hiraStr = kanaStr['hira']
    if hiraStr and isEnglish(hiraStr):
      # Entire phrase is in English
      hiraStr = translator.translate(hiraStr,dest='ja').text
      hiraStr = re.sub(r'[^\w\s]','',hiraStr)
      for kanaStrSub in kks.convert(hiraStr):
        hiraStrSub = kanaStrSub['hira']
        resStr += hiraStrSub
    else:
      resStr += hiraStr
    #print(kanaStr['hira'])
    #print(resStr)
    #break
  return resStr

def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

def punctPreprocess(text):
  punctList = ['！','？','、','!','?',',','(',')']
  for punct in punctList:
    text = text.replace(punct,'')
  return text

def englishFilter(phrase):
  if phrase and isEnglish(phrase):
    return translator.translate(phrase,dest='ja').text
  return phrase

In [ ]:

def extractLyrics(URL):
  
  #print(URL)
  r = requests.get(URL) 
  soup = BeautifulSoup(r.content, 'html5lib') 

  r = requests.get(URL) 
  table = soup.find('div', attrs = {'class':'content'})
  if table is None:
    return  
  #fileObj = open(filePath,"w") 

  resList = []
  for i,row in enumerate(table.findAll('p')):
    #print('------------ Verse ',i,' -------------')
    lyricText = lyricProcess(row.text) + '\n'
    #print(lyricText)
    for lyricLine in lyricText.split('\n'):
      #lyricLine.replace(u"\u3000","")
      if lyricLine == '':
        continue
      #resList.append(convertToKana(lyricLine))
      lyricLine = englishFilter(lyricLine)
      lyricLine = punctPreprocess(lyricLine)
      resList.append(lyricLine)   
      
    '''   
    print('----')
    
    for row in lyricText.split('\n'):
      row = convertToKana(row)
      print(row, ' ', len(row))
    '''
  #fileObj.close()
  return resList


#####################


def extractLyricTextFile(path):
  resList = []
  fileLyric = open(path, "r")

  #for lyricLine in lyricText.split('\n'):
  for lyricLine in fileLyric:
    
    lyricLine = lyricLine.replace('\n','')
    
    #lyricLine.replace(u"\u3000","")
    if lyricLine == '':
      continue
    #resList.append(convertToKana(lyricLine))
    lyricLine = englishFilter(lyricLine)
    lyricLine = punctPreprocess(lyricLine)
    #print(lyricLine)
    resList.append(lyricLine)

    #fileObj.writelines(lyricText) 
    
  
    #print('----')
  '''
  for lyricLine in resList:
    row = convertToKana(lyricLine)
    print(row, ' ', len(row))
  '''
  return resList


'''
URL = "http://stage48.net/studio48/areak.html"
lyricList = extractLyrics(URL)
'''

path = './lyric-data/Ya.txt'
lyricList = extractLyricTextFile(path)


In [ ]:
numTokenList = []
for i,lyricLine in enumerate(lyricList):

  lyricLength = len(convertToKana(lyricLine))
  numTokenList.append(lyricLength)
  #print(lyricLine,' ',lyricLength)

In [ ]:
print(lyricList)
print(numTokenList)

#numToken += str(lyricLength)+'|'

['世界の涯はて\u3000君は覚えているか', 'いつか話しただろう\u3000故郷ふるさとだ', '長距離バスの窓に陽が差し始めて', '肩を揺らすように起こされた', '疲れ切った身体からだをシートから立てて', '何年かぶりの海を見た', 'もう\u3000帰らないって', 'そう\u3000誓ったはずなのに', '夢は終わる', '世界の涯はて\u3000風は伝えているか', '波音の悲しみを…\u3000Oh Oh Yeah Yeah', '世界の涯はて\u3000君は信じているか', '愛は永遠だと…\u3000フェードアウト', '最後に旅をしよう\u3000僕が言い出して', '君をあの都会まちから連れて来た', '地図の上に何も書かれていない', '行きたい場所は空白だ', 'ねえ\u3000どこなのって', 'そう\u3000ここからその先は', '行き止まりさ', '心の涯はて\u3000光\u3000届いているか', '思い出は影になる\u3000Oh Oh Yeah Yeah', '心の涯はて\u3000僕を許してくれるか', '愛に未来はない\u3000ブラックアウト', '君に何を見せたかったか', '僕が生まれて\u3000育った岬で', '死ねたらいいねなんて…', '呟いてみたけど', '全部\u3000嘘さ', '涙の涯はて\u3000海に流れて行くか', '運命に悔いはない\u3000Oh Oh Yeah Yeah', '世界の涯はて\u3000風は伝えているか', '波音の悲しみを…\u3000Oh Oh Yeah Yeah', '世界の涯はて\u3000君は信じているか', '愛は永遠だと…\u3000フェードアウト']
[19, 20, 21, 14, 22, 13, 10, 12, 6, 19, 27, 19, 18, 19, 17, 17, 13, 9, 12, 6, 20, 26, 20, 17, 13, 16, 11, 8, 7, 19, 26, 19, 27, 19, 18]


In [ ]:
def createDataset(lyricList,numTokenList,printRes=False):
  dfListCurr = []
  dfListTar = []

  seqLen = 8
  stride = 2


  currInd = 0

  for i in range(0,math.ceil(len(lyricList)/seqLen)):
    currSeq = lyricList[currInd:(currInd+seqLen)]
    currStr = '|'.join(currSeq)
    dfListCurr.append(currStr)
    
    currSeq = numTokenList[currInd:(currInd+seqLen)]
    currStr = '|'.join(map(str, currSeq) )
    dfListTar.append(currStr)


    tarInd = currInd+seqLen
    currInd += stride
  # Add a piece of code to add the remaining parts

  currSeq = '|'.join(lyricList[(len(lyricList)-seqLen):])
  dfListCurr.append(currSeq)
  currSeq = '|'.join(map(str,numTokenList[(len(lyricList)-seqLen):]))
  dfListTar.append(currSeq)




  resLyrics = []
  resListTar = []
  for i,lyrics in enumerate(dfListCurr):
    if printRes:
      print(dfListTar[i])
      print(lyrics)
      print('\n\n')
    resListTar.append(dfListTar[i])
    resLyrics.append(lyrics)
  
  return resListTar, resLyrics

resListTar, resLyrics = createDataset(lyricList,numTokenList,True)
print(resListTar)
print(resLyrics)

19|20|21|14|22|13|10|12
世界の涯はて　君は覚えているか|いつか話しただろう　故郷ふるさとだ|長距離バスの窓に陽が差し始めて|肩を揺らすように起こされた|疲れ切った身体からだをシートから立てて|何年かぶりの海を見た|もう　帰らないって|そう　誓ったはずなのに



21|14|22|13|10|12|6|19
長距離バスの窓に陽が差し始めて|肩を揺らすように起こされた|疲れ切った身体からだをシートから立てて|何年かぶりの海を見た|もう　帰らないって|そう　誓ったはずなのに|夢は終わる|世界の涯はて　風は伝えているか



22|13|10|12|6|19|27|19
疲れ切った身体からだをシートから立てて|何年かぶりの海を見た|もう　帰らないって|そう　誓ったはずなのに|夢は終わる|世界の涯はて　風は伝えているか|波音の悲しみを…　Oh Oh Yeah Yeah|世界の涯はて　君は信じているか



10|12|6|19|27|19|18|19
もう　帰らないって|そう　誓ったはずなのに|夢は終わる|世界の涯はて　風は伝えているか|波音の悲しみを…　Oh Oh Yeah Yeah|世界の涯はて　君は信じているか|愛は永遠だと…　フェードアウト|最後に旅をしよう　僕が言い出して



6|19|27|19|18|19|17|17
夢は終わる|世界の涯はて　風は伝えているか|波音の悲しみを…　Oh Oh Yeah Yeah|世界の涯はて　君は信じているか|愛は永遠だと…　フェードアウト|最後に旅をしよう　僕が言い出して|君をあの都会まちから連れて来た|地図の上に何も書かれていない



8|7|19|26|19|27|19|18
呟いてみたけど|全部　嘘さ|涙の涯はて　海に流れて行くか|運命に悔いはない　Oh Oh Yeah Yeah|世界の涯はて　風は伝えているか|波音の悲しみを…　Oh Oh Yeah Yeah|世界の涯はて　君は信じているか|愛は永遠だと…　フェードアウト



['19|20|21|14|22|13|10|12', '21|14|22|13|10|12|6|19', '22|13|10|12|6|19|27|19', '10|12|6|19|27|19|18|19', '6|19|27|19|18|19|17|17', '8

In [ ]:
def splitPhrase(lyric):

  parseList = kks.convert(lyric)

  phraseLen = len(parseList)
  splitInt = int(phraseLen/2)

  if phraseLen == 1:
    resStr = parseList[0]['orig']
    tokenNumStr = str(len(convertToKana(resStr))) +'|'
    return resStr+'|', tokenNumStr

  resList = []
  resStr = ''
  tokenNumStr = ''

  currPhrase = ''
  for i,kanjiSet in enumerate(parseList):

    if i == splitInt:
      tokenNumStr += str(len(convertToKana(currPhrase))) + '|'
      resStr += currPhrase + '|'
      currPhrase = ''
        
    currPhrase += kanjiSet['orig']

  resStr += currPhrase + '|'
  tokenNumStr += str(len(convertToKana(currPhrase)))  + '|'

  return resStr, tokenNumStr

In [ ]:
# We now augment data by splitting each single phrase token in half

def splitLyricHalf(lyricList,numTokenList):
  splitNum = 8

  tokenNumStr = ''
  phraseStr = ''
  tokenLen = 0

  halfLyricList = []
  halfNumTokenList = []
  #print(lyricList)
  for i,lyric in enumerate(lyricList):
    #print('------',lyric,'---------')
    resStr, tokenNumSub = splitPhrase(lyric)
    #print(resStr)
    #print(tokenNumSub)
    #print(phraseStr)
    numOfToken = max(resStr.count('|'),1)

    if i% splitNum == 0:
      # We just ignore the new token then start the split
      '''
      print('< '+phraseStr+' >')
      print('< '+tokenNumStr+' >')
      print('___________________________')
      '''
      
      halfLyricList.append(phraseStr)
      halfNumTokenList.append(tokenNumStr)

      tokenNumStr = tokenNumSub
      phraseStr = resStr 
      tokenLen = numOfToken
      #print('')
      continue

    
    
    if tokenNumStr.count('|') + numOfToken > splitNum:
      # We just ignore the new token then start the split
      '''
      print('< '+phraseStr+' >')
      print('< '+tokenNumStr+' >')
      print('___________________________')
      '''
      halfLyricList.append(phraseStr)
      halfNumTokenList.append(tokenNumStr)

      tokenNumStr = tokenNumSub
      phraseStr = resStr 
      tokenLen = numOfToken
    else:
      tokenLen += numOfToken

      phraseStr += resStr 
      tokenNumStr += tokenNumSub

  #print('')
  resStr, tokenNumSub = splitPhrase(lyric)
  halfLyricList.append(phraseStr)
  halfNumTokenList.append(tokenNumStr)

  halfLyricStr = ''.join(halfLyricList).split('|')
  halfNumTokenStr = ''.join(halfNumTokenList).split('|')

  return halfLyricStr, halfNumTokenStr

halfLyricStr,halfNumTokenStr = splitLyricHalf(lyricList,numTokenList)

In [ ]:

print(halfLyricStr)
print(halfNumTokenStr)

['世界の涯はて\u3000', '君は覚えているか', 'いつか話し', 'ただろう\u3000故郷ふるさとだ', '長距離バスの窓に', '陽が差し始めて', '肩を揺ら', 'すように起こされた', '疲れ切った', '身体からだをシートから立てて', '何年かぶりの', '海を見た', 'もう\u3000', '帰らないって', 'そう\u3000', '誓ったはずなのに', '夢は', '終わる', '世界の涯はて\u3000', '風は伝えているか', '波音の', '悲しみを…\u3000Oh Oh Yeah Yeah', '世界の涯はて\u3000', '君は信じているか', '愛は', '永遠だと…\u3000フェードアウト', '最後に旅をしよう\u3000', '僕が言い出して', '君をあの都会', 'まちから連れて来た', '地図の上に', '何も書かれていない', '行きたい場所', 'は空白だ', 'ねえ\u3000どこなのって', 'そう\u3000ここからその', '先は', '行き止まり', 'さ', '心の涯はて\u3000', '光\u3000届いているか', '思い出は', '影になる\u3000Oh Oh Yeah Yeah', '心の涯はて\u3000', '僕を許してくれるか', '愛に', '未来はない\u3000ブラックアウト', '君に', '何を見せたかったか', '僕が生まれて\u3000', '育った岬で', '死ね', 'たらいいねなんて…', '呟', 'いてみたけど', '全部\u3000', '嘘さ', '涙の涯はて\u3000海', 'に流れて行くか', '運命に', '悔いはない\u3000Oh Oh Yeah Yeah', '世界の涯はて\u3000', '風は伝えているか', '波音の', '悲しみを…\u3000Oh Oh Yeah Yeah', '世界の涯はて\u3000', '君は信じているか', '愛は', '永遠だと…\u3000フェードアウト', '']
['9', '10', '6', '14', '12', '9', '5', '9', '6', '16', '8', '5', '3', '7', '3', '9', '3', '3', '9', '10', '5', '22

In [ ]:
resListTarHalf, resLyricsHalf = createDataset(halfLyricStr,halfNumTokenStr,True)
print(resListTarHalf)
print(resLyricsHalf)

9|10|6|14|12|9|5|9
世界の涯はて　|君は覚えているか|いつか話し|ただろう　故郷ふるさとだ|長距離バスの窓に|陽が差し始めて|肩を揺ら|すように起こされた



6|14|12|9|5|9|6|16
いつか話し|ただろう　故郷ふるさとだ|長距離バスの窓に|陽が差し始めて|肩を揺ら|すように起こされた|疲れ切った|身体からだをシートから立てて



12|9|5|9|6|16|8|5
長距離バスの窓に|陽が差し始めて|肩を揺ら|すように起こされた|疲れ切った|身体からだをシートから立てて|何年かぶりの|海を見た



5|9|6|16|8|5|3|7
肩を揺ら|すように起こされた|疲れ切った|身体からだをシートから立てて|何年かぶりの|海を見た|もう　|帰らないって



6|16|8|5|3|7|3|9
疲れ切った|身体からだをシートから立てて|何年かぶりの|海を見た|もう　|帰らないって|そう　|誓ったはずなのに



8|5|3|7|3|9|3|3
何年かぶりの|海を見た|もう　|帰らないって|そう　|誓ったはずなのに|夢は|終わる



3|7|3|9|3|3|9|10
もう　|帰らないって|そう　|誓ったはずなのに|夢は|終わる|世界の涯はて　|風は伝えているか



3|9|3|3|9|10|5|22
そう　|誓ったはずなのに|夢は|終わる|世界の涯はて　|風は伝えているか|波音の|悲しみを…　Oh Oh Yeah Yeah



3|3|9|10|5|22|9|10
夢は|終わる|世界の涯はて　|風は伝えているか|波音の|悲しみを…　Oh Oh Yeah Yeah|世界の涯はて　|君は信じているか



10|5|22|9|10|3|15|
風は伝えているか|波音の|悲しみを…　Oh Oh Yeah Yeah|世界の涯はて　|君は信じているか|愛は|永遠だと…　フェードアウト|



['9|10|6|14|12|9|5|9', '6|14|12|9|5|9|6|16', '12|9|5|9|6|16|8|5', '5|9|6|16|8|5|3|7', '6|16|8|5|3|7|3|9', '8|5|3|7|3|9|3|3', '3|7|3|9|3|3|9|10', '3|9|3|3|9|10|5|22', '3|3|9|10|5|2

Now we augment the data by double length of tokens

In [ ]:
print(lyricList)
print(numTokenList)

['世界の涯はて\u3000君は覚えているか', 'いつか話しただろう\u3000故郷ふるさとだ', '長距離バスの窓に陽が差し始めて', '肩を揺らすように起こされた', '疲れ切った身体からだをシートから立てて', '何年かぶりの海を見た', 'もう\u3000帰らないって', 'そう\u3000誓ったはずなのに', '夢は終わる', '世界の涯はて\u3000風は伝えているか', '波音の悲しみを…\u3000Oh Oh Yeah Yeah', '世界の涯はて\u3000君は信じているか', '愛は永遠だと…\u3000フェードアウト', '最後に旅をしよう\u3000僕が言い出して', '君をあの都会まちから連れて来た', '地図の上に何も書かれていない', '行きたい場所は空白だ', 'ねえ\u3000どこなのって', 'そう\u3000ここからその先は', '行き止まりさ', '心の涯はて\u3000光\u3000届いているか', '思い出は影になる\u3000Oh Oh Yeah Yeah', '心の涯はて\u3000僕を許してくれるか', '愛に未来はない\u3000ブラックアウト', '君に何を見せたかったか', '僕が生まれて\u3000育った岬で', '死ねたらいいねなんて…', '呟いてみたけど', '全部\u3000嘘さ', '涙の涯はて\u3000海に流れて行くか', '運命に悔いはない\u3000Oh Oh Yeah Yeah', '世界の涯はて\u3000風は伝えているか', '波音の悲しみを…\u3000Oh Oh Yeah Yeah', '世界の涯はて\u3000君は信じているか', '愛は永遠だと…\u3000フェードアウト']
[19, 20, 21, 14, 22, 13, 10, 12, 6, 19, 27, 19, 18, 19, 17, 17, 13, 9, 12, 6, 20, 26, 20, 17, 13, 16, 11, 8, 7, 19, 26, 19, 27, 19, 18]


In [ ]:
def formLyricDouble(lyricList,numTokenList,showRes=False):
  doubleLyricList = []
  doubleNumTokenList = []

  lyricTmp = ''
  numTmp = 0

  for i,lyric in enumerate(lyricList):
    #print('lyric: ',i)
    #print(len(lyricList))
    #print(len(numTokenList))

    if (i+1)%2 == 0:
      doubleLyricList.append(lyricTmp+lyric)
      doubleNumTokenList.append(numTmp+numTokenList[i])
      lyricTmp = ''
      numTmp = 0
    numTmp = numTokenList[i]
    lyricTmp = lyric

  if lyricTmp:
    doubleLyricList.append(lyricTmp)
    doubleNumTokenList.append(numTmp)
  if showRes:
    print(doubleLyricList)
    print(doubleNumTokenList)
  return doubleLyricList, doubleNumTokenList

doubleLyricList, doubleNumTokenList = formLyricDouble(lyricList,numTokenList,True)

['世界の涯はて\u3000君は覚えているかいつか話しただろう\u3000故郷ふるさとだ', '長距離バスの窓に陽が差し始めて肩を揺らすように起こされた', '疲れ切った身体からだをシートから立てて何年かぶりの海を見た', 'もう\u3000帰らないってそう\u3000誓ったはずなのに', '夢は終わる世界の涯はて\u3000風は伝えているか', '波音の悲しみを…\u3000Oh Oh Yeah Yeah世界の涯はて\u3000君は信じているか', '愛は永遠だと…\u3000フェードアウト最後に旅をしよう\u3000僕が言い出して', '君をあの都会まちから連れて来た地図の上に何も書かれていない', '行きたい場所は空白だねえ\u3000どこなのって', 'そう\u3000ここからその先は行き止まりさ', '心の涯はて\u3000光\u3000届いているか思い出は影になる\u3000Oh Oh Yeah Yeah', '心の涯はて\u3000僕を許してくれるか愛に未来はない\u3000ブラックアウト', '君に何を見せたかったか僕が生まれて\u3000育った岬で', '死ねたらいいねなんて…呟いてみたけど', '全部\u3000嘘さ涙の涯はて\u3000海に流れて行くか', '運命に悔いはない\u3000Oh Oh Yeah Yeah世界の涯はて\u3000風は伝えているか', '波音の悲しみを…\u3000Oh Oh Yeah Yeah世界の涯はて\u3000君は信じているか', '愛は永遠だと…\u3000フェードアウト']
[39, 35, 35, 22, 25, 46, 37, 34, 22, 18, 46, 37, 29, 19, 26, 45, 46, 18]


In [ ]:
resListTarDouble, resLyricsDouble = createDataset(doubleLyricList,doubleNumTokenList,True)
print(resListTarDouble)
print(resLyricsDouble)

39|35|35|22|25|46|37|34
世界の涯はて　君は覚えているかいつか話しただろう　故郷ふるさとだ|長距離バスの窓に陽が差し始めて肩を揺らすように起こされた|疲れ切った身体からだをシートから立てて何年かぶりの海を見た|もう　帰らないってそう　誓ったはずなのに|夢は終わる世界の涯はて　風は伝えているか|波音の悲しみを…　Oh Oh Yeah Yeah世界の涯はて　君は信じているか|愛は永遠だと…　フェードアウト最後に旅をしよう　僕が言い出して|君をあの都会まちから連れて来た地図の上に何も書かれていない



35|22|25|46|37|34|22|18
疲れ切った身体からだをシートから立てて何年かぶりの海を見た|もう　帰らないってそう　誓ったはずなのに|夢は終わる世界の涯はて　風は伝えているか|波音の悲しみを…　Oh Oh Yeah Yeah世界の涯はて　君は信じているか|愛は永遠だと…　フェードアウト最後に旅をしよう　僕が言い出して|君をあの都会まちから連れて来た地図の上に何も書かれていない|行きたい場所は空白だねえ　どこなのって|そう　ここからその先は行き止まりさ



25|46|37|34|22|18|46|37
夢は終わる世界の涯はて　風は伝えているか|波音の悲しみを…　Oh Oh Yeah Yeah世界の涯はて　君は信じているか|愛は永遠だと…　フェードアウト最後に旅をしよう　僕が言い出して|君をあの都会まちから連れて来た地図の上に何も書かれていない|行きたい場所は空白だねえ　どこなのって|そう　ここからその先は行き止まりさ|心の涯はて　光　届いているか思い出は影になる　Oh Oh Yeah Yeah|心の涯はて　僕を許してくれるか愛に未来はない　ブラックアウト



46|37|29|19|26|45|46|18
心の涯はて　光　届いているか思い出は影になる　Oh Oh Yeah Yeah|心の涯はて　僕を許してくれるか愛に未来はない　ブラックアウト|君に何を見せたかったか僕が生まれて　育った岬で|死ねたらいいねなんて…呟いてみたけど|全部　嘘さ涙の涯はて　海に流れて行くか|運命に悔いはない　Oh Oh Yeah Yeah世界の涯はて　風は伝えているか|波音の悲しみを…　Oh Oh Yeah Yeah世界の涯はて　君は信じているか|愛は永

Now we save the data into a csv file

In [ ]:
print(resListTar)
print(resLyrics)
print(resListTarHalf)
print(resLyricsHalf)
print(resListTarDouble)
print(resLyricsDouble)

['19|20|21|14|22|13|10|12', '21|14|22|13|10|12|6|19', '22|13|10|12|6|19|27|19', '10|12|6|19|27|19|18|19', '6|19|27|19|18|19|17|17', '8|7|19|26|19|27|19|18']
['世界の涯はて\u3000君は覚えているか|いつか話しただろう\u3000故郷ふるさとだ|長距離バスの窓に陽が差し始めて|肩を揺らすように起こされた|疲れ切った身体からだをシートから立てて|何年かぶりの海を見た|もう\u3000帰らないって|そう\u3000誓ったはずなのに', '長距離バスの窓に陽が差し始めて|肩を揺らすように起こされた|疲れ切った身体からだをシートから立てて|何年かぶりの海を見た|もう\u3000帰らないって|そう\u3000誓ったはずなのに|夢は終わる|世界の涯はて\u3000風は伝えているか', '疲れ切った身体からだをシートから立てて|何年かぶりの海を見た|もう\u3000帰らないって|そう\u3000誓ったはずなのに|夢は終わる|世界の涯はて\u3000風は伝えているか|波音の悲しみを…\u3000Oh Oh Yeah Yeah|世界の涯はて\u3000君は信じているか', 'もう\u3000帰らないって|そう\u3000誓ったはずなのに|夢は終わる|世界の涯はて\u3000風は伝えているか|波音の悲しみを…\u3000Oh Oh Yeah Yeah|世界の涯はて\u3000君は信じているか|愛は永遠だと…\u3000フェードアウト|最後に旅をしよう\u3000僕が言い出して', '夢は終わる|世界の涯はて\u3000風は伝えているか|波音の悲しみを…\u3000Oh Oh Yeah Yeah|世界の涯はて\u3000君は信じているか|愛は永遠だと…\u3000フェードアウト|最後に旅をしよう\u3000僕が言い出して|君をあの都会まちから連れて来た|地図の上に何も書かれていない', '呟いてみたけど|全部\u3000嘘さ|涙の涯はて\u3000海に流れて行くか|運命に悔いはない\u3000Oh Oh Yeah Yeah|世界の涯はて\u3000風は伝えているか|波音の悲しみを…\u3000Oh Oh Yeah Yeah|世界の

In [ ]:
resListFull = resListTar + resListTarHalf + resListTarDouble
resLyricFull = resLyrics + resLyricsHalf + resLyricsDouble
print(resListFull)
print(resLyricFull)
print(len(resListFull))
print(len(resLyricFull))

['19|20|21|14|22|13|10|12', '21|14|22|13|10|12|6|19', '22|13|10|12|6|19|27|19', '10|12|6|19|27|19|18|19', '6|19|27|19|18|19|17|17', '8|7|19|26|19|27|19|18', '9|10|6|14|12|9|5|9', '6|14|12|9|5|9|6|16', '12|9|5|9|6|16|8|5', '5|9|6|16|8|5|3|7', '6|16|8|5|3|7|3|9', '8|5|3|7|3|9|3|3', '3|7|3|9|3|3|9|10', '3|9|3|3|9|10|5|22', '3|3|9|10|5|22|9|10', '10|5|22|9|10|3|15|', '39|35|35|22|25|46|37|34', '35|22|25|46|37|34|22|18', '25|46|37|34|22|18|46|37', '46|37|29|19|26|45|46|18']
['世界の涯はて\u3000君は覚えているか|いつか話しただろう\u3000故郷ふるさとだ|長距離バスの窓に陽が差し始めて|肩を揺らすように起こされた|疲れ切った身体からだをシートから立てて|何年かぶりの海を見た|もう\u3000帰らないって|そう\u3000誓ったはずなのに', '長距離バスの窓に陽が差し始めて|肩を揺らすように起こされた|疲れ切った身体からだをシートから立てて|何年かぶりの海を見た|もう\u3000帰らないって|そう\u3000誓ったはずなのに|夢は終わる|世界の涯はて\u3000風は伝えているか', '疲れ切った身体からだをシートから立てて|何年かぶりの海を見た|もう\u3000帰らないって|そう\u3000誓ったはずなのに|夢は終わる|世界の涯はて\u3000風は伝えているか|波音の悲しみを…\u3000Oh Oh Yeah Yeah|世界の涯はて\u3000君は信じているか', 'もう\u3000帰らないって|そう\u3000誓ったはずなのに|夢は終わる|世界の涯はて\u3000風は伝えているか|波音の悲しみを…\u3000Oh Oh Yeah Yeah|世界の涯はて\u3000君は信じているか|愛は永遠だと…

We create dataset for all songs

In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
!rm entireNotes.csv

In [ ]:
!rm entireNotes.csv

lyricDir = './lyric-data/'
resListFull = []
resLyricFull = []

totalLineNum = 0

for i,fileLyric in enumerate(os.listdir(lyricDir)):
  print(i,' : ',fileLyric)
  fullLyricPath = lyricDir + fileLyric
  lyricList = extractLyricTextFile(fullLyricPath)
  numTokenList = []
  for j,lyricLine in enumerate(lyricList):

    lyricLength = len(convertToKana(lyricLine))
    numTokenList.append(lyricLength)


  resListTar, resLyrics = createDataset(lyricList,numTokenList)

  resListTarHalf, resLyricsHalf = splitLyricHalf(lyricList,numTokenList)
  resListTarHalf, resLyricsHalf = createDataset(resListTarHalf,resLyricsHalf)

  doubleLyricList, doubleNumTokenList = formLyricDouble(lyricList,numTokenList)
  resListTarDouble, resLyricsDouble = createDataset(doubleLyricList,doubleNumTokenList)

  resListFull += resListTar + resListTarHalf + resListTarDouble
  resLyricFull += resLyrics + resLyricsHalf + resLyricsDouble
  

  dfsrc = pd.DataFrame(resListFull)
  df2trg = pd.DataFrame(resLyricFull)

  

  frames = [dfsrc, df2trg]
  df = pd.concat(frames,axis=1)

  df.to_csv("entireNotes.csv", index=False,mode='a', header=False)

  totalLineNum += len(resListFull)

  #print(resListTarHalf)
  #print(doubleLyricList)
  print('   len: ',len(resListFull),'  : ',totalLineNum)

  resListFull = []  
  resLyricFull = []



0  :  Tomo-yo.txt
   len:  22   :  22
1  :  Class-kai-no-ato-de.txt
   len:  25   :  47
2  :  I'm-free.txt
   len:  30   :  77
3  :  Dark-Side.txt
   len:  41   :  118
4  :  Hitonatsu-no-nagasa-yori.txt
   len:  27   :  145
5  :  Pinocchio-gun-SKE48-Stage-Version.txt
   len:  21   :  166
6  :  Good-Timing.txt
   len:  26   :  192
7  :  Rappa-renshuuchuu.txt
   len:  23   :  215
8  :  Hatsukoi-Door.txt
   len:  24   :  239
9  :  So~Yu~Toko-aru-yo-ne.txt
   len:  37   :  276
10  :  Bokutachi-wa-Sinbad-da.txt
   len:  24   :  300
11  :  FRUSTRATION.txt
   len:  47   :  347
12  :  Konya-wa-Shake-it.txt
   len:  48   :  395
13  :  Tadashii-mahou-no-tsukaikata.txt
   len:  26   :  421
14  :  Tenohira-Deguchi-Aki-Album-Version.txt
   len:  20   :  441
15  :  Mimi-ni-ochiru-namida.txt
   len:  22   :  463
16  :  Kyouhansha.txt
   len:  27   :  490
17  :  Maenomeri.txt
   len:  28   :  518
18  :  BOW-WOW.txt
   len:  22   :  540
19  :  Sakasazaka.txt
   len:  16   :  556
20  :  Hetappi-Wink.txt

In [ ]:


'''
dfsrc = pd.DataFrame(resListFull)
df2trg = pd.DataFrame(resLyricFull)

frames = [dfsrc, df2trg]
df = pd.concat(frames,axis=1)

df.to_csv("entireNotes.csv", index=False)
#print(df)
'''
msk = np.random.rand(len(df)) < 0.8
train_df = df[msk]
test_df = df[~msk]
#print(test_df)

train_df.to_csv("trainNotes.csv", index=False)
test_df.to_csv("valNotes.csv", index=False)


In [ ]:
'''
tmp = '目の前に広がった道は'

resStr = ''
for kanaStr in kks.convert(tmp):
  tmpRes += kanaStr['hira']
print(tmpRes)
'''

"\ntmp = '目の前に広がった道は'\n\nresStr = ''\nfor kanaStr in kks.convert(tmp):\n  tmpRes += kanaStr['hira']\nprint(tmpRes)\n"

In [ ]:
'''
#word = '目の前に広がった道は'
word =  'OH〜　Brother!'
print(isEnglish(word))
'''

"\n#word = '目の前に広がった道は'\nword =  'OH〜\u3000Brother!'\nprint(isEnglish(word))\n"

In [ ]:
'''
#res = translator.translate('目の前に広がった道は')
res = translator.translate('OH〜　Brother!',dest='ja')
#res = translator.translate('Everybody dance!',dest='ja')
print(res.text)
'''

"\n#res = translator.translate('目の前に広がった道は')\nres = translator.translate('OH〜\u3000Brother!',dest='ja')\n#res = translator.translate('Everybody dance!',dest='ja')\nprint(res.text)\n"